In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from utils.preprocess import get_df, set_data_for_model

In [2]:
data_path = 'data/data.csv'
df = get_df(data_path)
X_train, X_valid, y_train, y_valid = set_data_for_model(df, 42)

y_train_one_hot = np.eye(2)[y_train]
y_valid_one_hot = np.eye(2)[y_valid]

In [3]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_one_hot, dtype=torch.float32)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid_one_hot, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)

In [4]:
class MLPModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLPModel, self).__init__()
        self.hidden1 = nn.Linear(input_size, 10)
        self.hidden2 = nn.Linear(10, 5)
        self.output = nn.Linear(5, output_size)
        self.activation = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.activation(self.hidden1(x))
        x = self.activation(self.hidden2(x))
        x = self.softmax(self.output(x))
        return x

In [5]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=100):
    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.argmax(dim=1))
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == targets.argmax(dim=1)).sum().item()
            total_train += targets.size(0)

        model.eval()
        val_loss = 0.0
        correct_valid = 0
        total_valid = 0
        with torch.no_grad():
            for inputs, targets in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets.argmax(dim=1))
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct_valid += (predicted == targets.argmax(dim=1)).sum().item()
                total_valid += targets.size(0)
        
        train_accuracy = correct_train / total_train
        val_accuracy = correct_valid / total_valid
        history['loss'].append(running_loss / len(train_loader))
        history['val_loss'].append(val_loss / len(valid_loader))
        history['accuracy'].append(train_accuracy)
        history['val_accuracy'].append(val_accuracy)

        print(f"Epoch {epoch+1}/{epochs} - loss: {history['loss'][-1]:.4f} - val_loss: {history['val_loss'][-1]:.4f} - accuracy: {train_accuracy:.4f} - val_accuracy: {val_accuracy:.4f}")
    
    return history

In [6]:
def evaluate_model(model, valid_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets.argmax(dim=1))
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets.argmax(dim=1)).sum().item()
            total += targets.size(0)
    
    val_loss /= len(valid_loader)
    val_accuracy = correct / total
    print(f"Validation loss: {val_loss:.4f}")
    print(f"Validation accuracy: {val_accuracy:.4f}")

In [7]:
input_size = X_train.shape[1]
output_size = 2
model_pytorch = MLPModel(input_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_pytorch.parameters(), lr=0.035)

history_pytorch = train_model(model_pytorch, train_loader, valid_loader, criterion, optimizer, epochs=100)
evaluate_model(model_pytorch, valid_loader, criterion)

Epoch 1/100 - loss: 0.7069 - val_loss: 0.6977 - accuracy: 0.2769 - val_accuracy: 0.4386
Epoch 2/100 - loss: 0.6896 - val_loss: 0.6847 - accuracy: 0.5582 - val_accuracy: 0.6053
Epoch 3/100 - loss: 0.6774 - val_loss: 0.6742 - accuracy: 0.6264 - val_accuracy: 0.6228
Epoch 4/100 - loss: 0.6680 - val_loss: 0.6652 - accuracy: 0.6286 - val_accuracy: 0.6228
Epoch 5/100 - loss: 0.6600 - val_loss: 0.6561 - accuracy: 0.6286 - val_accuracy: 0.6228
Epoch 6/100 - loss: 0.6518 - val_loss: 0.6454 - accuracy: 0.6286 - val_accuracy: 0.6228
Epoch 7/100 - loss: 0.6387 - val_loss: 0.6320 - accuracy: 0.6330 - val_accuracy: 0.6491
Epoch 8/100 - loss: 0.6287 - val_loss: 0.6148 - accuracy: 0.6549 - val_accuracy: 0.7193
Epoch 9/100 - loss: 0.6091 - val_loss: 0.5937 - accuracy: 0.7099 - val_accuracy: 0.7632
Epoch 10/100 - loss: 0.5919 - val_loss: 0.5677 - accuracy: 0.7692 - val_accuracy: 0.8246
Epoch 11/100 - loss: 0.5707 - val_loss: 0.5381 - accuracy: 0.8484 - val_accuracy: 0.9123
Epoch 12/100 - loss: 0.5378 - 